# 이 노트의 목표 (240413)
---
- 데이터 원본 그대로 split 할건데, 아래 두 가지중 하나로 결정. (df로 분석)
    1. duration 따라서
    2. annotation 정보 따라서

---
해보니까 duration대로 잘라도 어노테이션 비율은 그대로 유지(대부분 20초짜리 데이터라서 그런듯), <br>
대신 train dataset에서 normal비율이 압도적으로 많음(75%)<br>
아마 어떻게 잘라도 같은 비율로 데이터가 만들어질 것 같음

대신 이 방법으로 얻은건, 같은 데이터 셋에 같은 환자 데이터가 들어간다는 것


In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
# import

import pandas as pd
import numpy as np
import librosa
import os
import random


In [ ]:
database_dir = '/content/drive/MyDrive/data/ICBHI_final_database/'
annotation_csv = '/content/drive/MyDrive/for_update/annotation.csv'
counting_csv = '/content/drive/MyDrive/for_update/label_counts.csv'

### data distribution 확인

In [ ]:
annotation_df = pd.read_csv(annotation_csv)
counting_df = pd.read_csv(counting_csv)

In [ ]:
annotation_df.head()

,filename,start,end,crackles,wheezes,both,normal
0,101_1b1_Al_sc_Meditron,0.036,0.579,0,0,0,1
1,101_1b1_Al_sc_Meditron,0.579,2.450,0,0,0,1
2,101_1b1_Al_sc_Meditron,2.450,3.893,0,0,0,1
3,101_1b1_Al_sc_Meditron,3.893,5.793,0,0,0,1
4,101_1b1_Al_sc_Meditron,5.793,7.521,0,0,0,1


In [ ]:
counting_df.head()

,filename,duration,crackles,wheezes,only_crackles,only_wheezes,both,normal,total
0,101_1b1_Al_sc_Meditron,20.000,0,0,0,0,0,12,12
1,101_1b1_Pr_sc_Meditron,20.000,0,0,0,0,0,11,11
2,102_1b1_Ar_sc_Meditron,20.000,0,0,0,0,0,13,13
3,103_2b2_Ar_mc_LittC2SE,20.000,0,4,0,4,0,2,6
4,104_1b1_Al_sc_Litt3200,15.856,0,0,0,0,0,6,6


In [ ]:
'''
고려할점:
스플릿할 때 같은 세트에 같은 환자 데이터가 들어가도록.. inference를 위해서
분포 고려해야하는건
레이블 개수? 랑 7로 나눴을 때 듀레이션?
'''

total_duration = counting_df['duration'].sum()
print(total_duration) # 19773초
train_volumn = total_duration*0.7
val_volumn = total_duration*0.1
test_volumn = total_duration*0.2

print(train_volumn, val_volumn, test_volumn)

'''
데이터 전체를 random(42) suffle해주고,
suffled data를 위에서부터 읽으면서 duration을 더해줌.
해당 split의 볼륨이 넘어가기 전에 다음 스플릿으로 넘겨주기(train/val/test column추가하는 방식으로)

그 다음 클래스 분포 체크해보기
'''
# 클래스 분포
total_c = counting_df['crackles'].sum()
total_w = counting_df['wheezes'].sum()
total_only_c = counting_df['only_crackles'].sum()
total_only_w = counting_df['only_wheezes'].sum()
total_n = counting_df['normal'].sum()
total_b = counting_df['both'].sum()

print(total_c, total_w, total_only_c, total_only_w, total_n, total_b)

# ratio
print(total_only_c + total_only_w + total_n + total_b) # 6898
print(total_only_c/6898, total_only_w/6898, total_n/6898, total_b/6898) # 2.7 : 1.3 : 5.3 : 0.7
print(total_c/6898, total_w/6898, total_n/6898) # 3:2:5

19773.032
13841.122399999998 1977.3032 3954.6064
2370 1392 1864 886 3642 506
6898
0.2702232531168455 0.1284430269643375 0.5279791243838794 0.07335459553493766
0.34357784865178315 0.20179762249927516 0.5279791243838794


### seed 설정

In [ ]:
shuffled_df = counting_df.sample(frac=1, random_state=42).reset_index(drop=True)
shuffled_df.head()

,filename,duration,crackles,wheezes,only_crackles,only_wheezes,both,normal,total
0,151_2p3_Pl_mc_AKGC417L,20.0,0,5,0,5,0,1,6
1,156_8b3_Lr_mc_AKGC417L,20.0,0,1,0,1,0,7,8
2,172_2b5_Pl_mc_AKGC417L,20.0,3,1,3,1,0,2,6
3,147_2b3_Lr_mc_AKGC417L,20.0,0,0,0,0,0,5,5
4,172_1b5_Lr_mc_AKGC417L,20.0,2,1,2,1,0,4,7


In [ ]:
shuffled_df.index[-1]
a = range(shuffled_df.index[-1]+1)
shuffled_df.loc[0]

filename         151_2p3_Pl_mc_AKGC417L
duration                           20.0
crackles                              0
wheezes                               5
only_crackles                         0
only_wheezes                          5
both                                  0
normal                                1
total                                 6
Name: 0, dtype: object

In [ ]:
shuffled_df.iloc[0]


filename         151_2p3_Pl_mc_AKGC417L
duration                           20.0
crackles                              0
wheezes                               5
only_crackles                         0
only_wheezes                          5
both                                  0
normal                                1
total                                 6
Name: 0, dtype: object

In [ ]:
shuffled_df['duration'][0]

20.0

In [ ]:
# train_volumn = total_duration*0.7
# val_volumn = total_duration*0.1
# test_volumn = total_duration*0.2
print(train_volumn, val_volumn, test_volumn)


13841.122399999998 1977.3032 3954.6064


In [ ]:
train_volumn, val_volumn, test_volumn

(13841.122399999998, 1977.3032, 3954.6064)

In [ ]:
'''
train_volumn = total_duration*0.7
val_volumn = total_duration*0.1
test_volumn = total_duration*0.2
'''


sum_duration = 0
split_ls = []
for i in range(shuffled_df.index[-1]+1):
    if sum_duration < train_volumn:
        split_ls.append('train')
    elif train_volumn < sum_duration and sum_duration <= (train_volumn + val_volumn):
        split_ls.append('val')
    elif (train_volumn + val_volumn) <= sum_duration:
        split_ls.append('test')
    sum_duration += shuffled_df['duration'][i]

print(sum_duration)


19773.032


In [ ]:
print(split_ls.count('train'))
print(split_ls.count('val'))
print(split_ls.count('test'))

print(split_ls.count('train')/920)
print(split_ls.count('val')/920)
print(split_ls.count('test')/920)


642
94
184
0.6978260869565217
0.10217391304347827
0.2


In [ ]:
shuffled_df['split'] = split_ls
shuffled_df.head()

,filename,duration,crackles,wheezes,only_crackles,only_wheezes,both,normal,total,split
0,151_2p3_Pl_mc_AKGC417L,20.0,0,5,0,5,0,1,6,train
1,156_8b3_Lr_mc_AKGC417L,20.0,0,1,0,1,0,7,8,train
2,172_2b5_Pl_mc_AKGC417L,20.0,3,1,3,1,0,2,6,train
3,147_2b3_Lr_mc_AKGC417L,20.0,0,0,0,0,0,5,5,train
4,172_1b5_Lr_mc_AKGC417L,20.0,2,1,2,1,0,4,7,train


In [ ]:
sorted_df = shuffled_df.sort_values(by='filename').reset_index(drop=True)
sorted_df.head()

,filename,duration,crackles,wheezes,only_crackles,only_wheezes,both,normal,total,split
0,101_1b1_Al_sc_Meditron,20.000,0,0,0,0,0,12,12,train
1,101_1b1_Pr_sc_Meditron,20.000,0,0,0,0,0,11,11,test
2,102_1b1_Ar_sc_Meditron,20.000,0,0,0,0,0,13,13,train
3,103_2b2_Ar_mc_LittC2SE,20.000,0,4,0,4,0,2,6,train
4,104_1b1_Al_sc_Litt3200,15.856,0,0,0,0,0,6,6,test


In [ ]:
train_df = shuffled_df[shuffled_df['split'] == 'train'].reset_index(drop=True)
val_df = shuffled_df[shuffled_df['split'] == 'val'].reset_index(drop=True)
test_df = shuffled_df[shuffled_df['split'] == 'test'].reset_index(drop=True)


In [ ]:
print(train_df.shape)
print(val_df.shape)
print(test_df.shape)


(642, 10)
(94, 10)
(184, 10)


In [ ]:
# 클래스 분포
train_c = train_df['crackles'].sum()
train_w = train_df['wheezes'].sum()
train_only_c = train_df['only_crackles'].sum()
train_only_w = train_df['only_wheezes'].sum()
train_n = train_df['normal'].sum()
train_b = train_df['both'].sum()

val_c = val_df['crackles'].sum()
val_w = val_df['wheezes'].sum()
val_only_c = val_df['only_crackles'].sum()
val_only_w = val_df['only_wheezes'].sum()
val_n = val_df['normal'].sum()
val_b = val_df['both'].sum()

test_c = test_df['crackles'].sum()
test_w = test_df['wheezes'].sum()
test_only_c = test_df['only_crackles'].sum()
test_only_w = test_df['only_wheezes'].sum()
test_n = test_df['normal'].sum()
test_b = test_df['both'].sum()

# 27 : 13 : 53 : 07
# 3:2:5

print('train:', train_c, train_w, train_only_c, train_only_w, train_n, train_b) # train: 1696 982 1346 632 2511 350
print('\t', train_only_c + train_only_w + train_n + train_b) # 4839
print('\t', total_only_c/4839*100, total_only_w/4839*100, total_n/4839*100, total_b/4839*100) # 2.7 : 1.3 : 5.3 : 0.7
print('\t', train_c/4839*100, train_w/4839*100, train_n/4839*100) # 3:2:5
print()

print('val:', val_c, val_w, val_only_c, val_only_w, val_n, val_b) # val: 216 132 161 77 381 55
print('\t', val_only_c + val_only_w + val_n + val_b) # 674
print('\t', val_only_c/674*100, val_only_w/674*100, val_n/674*100, val_b/674*100) # 2.7 : 1.3 : 5.3 : 0.7
print('\t', val_c/674*100, val_w/674*100, val_n/674*100) # 3:2:5
print()
print('test:', test_c, test_w, test_only_c, test_only_w, test_n, test_b) # test: 458 278 357 177 750 101
print('\t', test_only_c + test_only_w + test_n + test_b) # 1385
print('\t', test_only_c/1385*100, test_only_w/1385*100, test_n/1385*100, test_b/1385*100) # 2.7 : 1.3 : 5.3 : 0.7
print('\t', test_c/1385*100, test_w/1385*100, test_n/1385*100) # 3:2:5
print()

'''
seed 42:
train: 1696 982 1346 632 2511 350
	 4839
	 38.52035544533995 18.30956809258111 75.26348419094855 10.456705930977476
	 35.048563752841495 20.293449059723084 51.89088654680719

val: 216 132 161 77 381 55
	 674
	 23.887240356083087 11.42433234421365 56.52818991097923 8.160237388724036
	 32.047477744807125 19.584569732937684 56.52818991097923

test: 458 278 357 177 750 101
	 1385
	 25.776173285198556 12.779783393501805 54.151624548736464 7.292418772563177
	 33.068592057761734 20.072202166064983 54.151624548736464
'''


train: 1696 982 1346 632 2511 350
	 4839
	 38.52035544533995 18.30956809258111 75.26348419094855 10.456705930977476
	 35.048563752841495 20.293449059723084 51.89088654680719

val: 216 132 161 77 381 55
	 674
	 23.887240356083087 11.42433234421365 56.52818991097923 8.160237388724036
	 32.047477744807125 19.584569732937684 56.52818991097923

test: 458 278 357 177 750 101
	 1385
	 25.776173285198556 12.779783393501805 54.151624548736464 7.292418772563177
	 33.068592057761734 20.072202166064983 54.151624548736464



'\nseed 42:\ntrain: 1696 982 1346 632 2511 350\n\t 4839\n\t 38.52035544533995 18.30956809258111 75.26348419094855 10.456705930977476\n\t 35.048563752841495 20.293449059723084 51.89088654680719\n\nval: 216 132 161 77 381 55\n\t 674\n\t 23.887240356083087 11.42433234421365 56.52818991097923 8.160237388724036\n\t 32.047477744807125 19.584569732937684 56.52818991097923\n\ntest: 458 278 357 177 750 101\n\t 1385\n\t 25.776173285198556 12.779783393501805 54.151624548736464 7.292418772563177\n\t 33.068592057761734 20.072202166064983 54.151624548736464\n'

In [ ]:
train_df.sort_values(by='filename').reset_index(drop=True).to_csv('/content/drive/MyDrive/for_update/train.csv', index=False)
val_df.sort_values(by='filename').reset_index(drop=True).to_csv('/content/drive/MyDrive/for_update/val.csv', index=False)
test_df.sort_values(by='filename').reset_index(drop=True).to_csv('/content/drive/MyDrive/for_update/test.csv', index=False)


In [ ]:
val_df.sort_values(by='filename').reset_index(drop=True)

,filename,duration,crackles,wheezes,only_crackles,only_wheezes,both,normal,total,split
0,130_1p2_Ar_mc_AKGC417L,20.00,7,4,3,0,4,0,7,val
1,130_1p3_Pl_mc_AKGC417L,20.00,0,5,0,5,0,1,6,val
2,130_1p3_Pr_mc_AKGC417L,20.00,0,2,0,2,0,4,6,val
3,130_1p4_Pl_mc_AKGC417L,20.00,0,6,0,6,0,1,7,val
4,130_2b3_Pl_mc_AKGC417L,20.00,9,0,9,0,0,0,9,val
...,...,...,...,...,...,...,...,...,...,...
89,218_1b1_Ar_sc_Meditron,71.05,13,12,2,1,11,2,16,val
90,220_1b2_Al_mc_LittC2SE,20.00,0,0,0,0,0,7,7,val
91,221_2b2_Al_mc_LittC2SE,20.00,0,5,0,5,0,1,6,val
92,221_2b3_Al_mc_LittC2SE,20.00,0,5,0,5,0,0,5,val


In [ ]:
sample = '130_1p2_Ar_mc_AKGC417L.txt' # 모든 조합이 포함된 annotation


tmp = pd.read_csv(os.path.join(database_dir, sample), sep='\t', header=None)
tmp

,0,1,2,3
0,0.697,2.423,1,0
1,2.423,5.101,1,0
2,5.101,8.339,1,1
3,8.339,11.280,1,0
4,11.280,14.316,1,1
5,14.316,17.030,1,1
6,17.030,19.554,1,1
